In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold, train_test_split
#import graphviz <-------------- 圖示化決策樹的
from sklearn import  preprocessing , metrics 
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor ,ExtraTreesRegressor
from xgboost import XGBRFRegressor 
import xgboost as xgb
from sklearn import svm

# Load data from the dataset




In [3]:
X = pd.read_csv('C:\\Users\\yunting\\Documents\\GitHub\\E.sun_conpetition\\data_set\\new\\2_combine_address_drop_parking_area\\ready_train_2.csv')
y = pd.read_csv('C:\\Users\\yunting\\Documents\\GitHub\\E.sun_conpetition\\data_set\\new\\ready_y.csv')

In [4]:
sub = pd.read_csv('C:\\Users\\yunting\\Documents\\GitHub\\E.sun_conpetition\\data_set\\initial\\submit_test.csv')
test = pd.read_csv('C:\\Users\\yunting\\Documents\\GitHub\\E.sun_conpetition\\data_set\\new\\2_combine_address_drop_parking_area\\ready_train_2.csv')

# Normalize the data

In [5]:
def normalization(X , method ) :
    if method == 'MinMaxScaler' :
        norm = preprocessing.MinMaxScaler()
        X_norm =  norm.fit_transform(X)
    if method == 'normalize' :
        X_norm = preprocessing.normalize(X,'l2')
    return X_norm

In [6]:
X_norm = normalization(X,'MinMaxScaler')
y_norm = normalization(y,'MinMaxScaler')
test_norm = normalization(test,'MinMaxScaler')

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


# Feature selection

In [7]:
import seaborn as sns
import eli5
from eli5.sklearn import PermutationImportance
from sklearn.feature_selection import SelectFromModel
from sklearn.decomposition import PCA

# use corr_matrix and eli5 to check feature ranking

In [8]:
trainX , testX , train_y , test_y = train_test_split (X , y, test_size = 0.25)
kf = KFold(n_splits=5)

In [9]:
my_model = xgb.XGBRegressor(n_estimators=400,subsample=0.88,max_depth=3,colsample_bytree=0.88,min_child_weight=1,reg_alpha=0.01,objective='reg:squarederror').fit(trainX, train_y)
perm = PermutationImportance(my_model, random_state=40).fit(testX, test_y)
eli5.show_weights(perm, feature_names = X.columns.tolist())

Weight,Feature
1.2074 ± 0.2567,building_area
0.3515 ± 0.0779,land_area
0.1427 ± 0.1106,III_10000
0.1049 ± 0.0359,parking_price
0.0497 ± 0.0301,II_10000
0.0459 ± 0.0286,village_income_median
0.0217 ± 0.0074,XIII_5000
0.0166 ± 0.0014,II_1000
0.0159 ± 0.0325,address_13171370
0.0149 ± 0.0004,XII_1000


# feature select form pca

In [10]:
pca = PCA(n_components=400)
fepca= pca.fit(X)
X_pca = fepca.transform(X)
test_pca = fepca.transform(test)

# 測試PCA效果

In [32]:
# full model
print(X.shape)
print('tree_without_pca',cross_val_score(treereg, X, y, cv=kf, n_jobs = -1))
print('forest_without_pca',cross_val_score(forest, X, y, cv=kf, n_jobs = -1))
print('extree_without_pca',cross_val_score(extree, X, y, cv=kf, n_jobs = -1))
print('xgb_without_pca',cross_val_score(xgbreg, X, y, cv=kf, n_jobs = -1))

(60000, 4498)
tree_without_pca [0.5021147  0.59227604 0.70609805 0.50322224 0.7836085 ]
forest_without_pca [0.66323202 0.79825209 0.83918539 0.54587361 0.82377066]
extree_without_pca [0.69478572 0.82455806 0.84025053 0.77789005 0.89386383]
xgb_without_pca [0.77883917 0.76535338 0.8529166  0.65583195 0.83396952]


In [33]:
# pca
print(X_pca.shape)
print('tree_with_pca',cross_val_score(treereg, X_pca, y, cv=kf, n_jobs = -1))
print('forest_with_pca',cross_val_score(forest, X_pca, y, cv=kf, n_jobs = -1))
print('extree_with_pca',cross_val_score(extree, X_pca, y, cv=kf, n_jobs = -1))      
print('xgb_with_pca',cross_val_score(xgbreg, X_pca, y, cv=kf, n_jobs = -1))

(60000, 400)
tree_with_pca [-0.58034541  0.05849113  0.26264679  0.55360754  0.46677066]
forest_with_pca [0.51427657 0.59688968 0.68427052 0.765469   0.78117552]
extree_with_pca [0.61133274 0.7941976  0.70438667 0.79784223 0.89000518]
xgb_with_pca [0.62117183 0.71878373 0.76283997 0.73153413 0.82937225]


# feature select form model

In [11]:
X.shape

(60000, 4498)

# select from randomforest

In [12]:
forest = RandomForestRegressor(n_estimators=400,n_jobs=-1)

forestreg = forest.fit(X,y)

model_forest = SelectFromModel(forestreg,prefit=True)
X_forest = model_forest.transform(X)
X_forest.shape

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


(60000, 147)

In [13]:
#forest = RandomForestRegressor(n_estimators=400,n_jobs=-1)

#forestreg_pca = forest.fit(X_pca,y)

#model_forest_pca = SelectFromModel(forestreg_pca,prefit=True)
#X_forest_pca = model_forest_pca.transform(X)
#X_forest_pca.shape

# select from Extratree

In [14]:
extree = ExtraTreesRegressor(n_estimators=200,n_jobs=-1)

extree_reg = extree.fit(X,y)

model_ex = SelectFromModel(extree_reg,prefit=True)
X_ex = model_ex.transform(X)
X_ex.shape

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


(60000, 207)

In [15]:
#extree = ExtraTreesRegressor(n_estimators=200,n_jobs=-1)

#extree_reg_pca = extree.fit(X_pca,y)

#model_ex_pca = SelectFromModel(extree_reg_pca,prefit=True)
#X_ex_pca = model_ex_pca.transform(X)
#X_ex_pca.shape

# 將選擇好的特徵 FILTER回 X 

In [16]:
x_forest = X[X.columns[model_forest.get_support()]]
x_ex = X[X.columns[model_ex.get_support()]]

In [17]:
mixforest = pd.concat([x_forest,test],axis=0).dropna(axis=1)
mixex = pd.concat([x_ex,test],axis=0).dropna(axis=1)
test_forest = mixforest.iloc[60000:,:]
test_ex = mixex.iloc[60000:,:]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


# 將降維度後的資料帶入Model 

# 測試select from model 效果

In [34]:
### 原始效果在PCA處

In [36]:
# forest baise
print(X_forest.shape)
print('tree_with_forestmodel',cross_val_score(treereg, X_forest, y, cv=kf, n_jobs = -1))
print('forest_with_forestmodel',cross_val_score(forest, X_forest, y, cv=kf, n_jobs = -1))
print('extree_without_pca',cross_val_score(extree, X_forest, y, cv=kf, n_jobs = -1))
print('xgb_with_forestmodel',cross_val_score(xgbreg, X_forest, y, cv=kf, n_jobs = -1))

(60000, 147)
tree_with_forestmodel [0.49162044 0.56759198 0.93765555 0.10964331 0.80966119]
forest_with_forestmodel [0.67779653 0.79894471 0.82524309 0.53459    0.83310469]
extree_without_pca [0.69435879 0.79862834 0.84512453 0.78041455 0.89235596]
xgb_with_forestmodel [0.80542269 0.76646104 0.85748394 0.70201472 0.84473071]


In [38]:
# extree baise
print(X_ex.shape)
print('tree_with_treemodel',cross_val_score(treereg, X_ex, y, cv=kf, n_jobs = -1))
print('forest_with_treemodel',cross_val_score(forest, X_ex, y, cv=kf, n_jobs = -1))
print('extree_with_treemodel',cross_val_score(extree, X_ex, y, cv=kf, n_jobs = -1))
print('xgb_with_treemodel',cross_val_score(xgbreg, X_ex, y, cv=kf, n_jobs = -1))

(60000, 207)
tree_with_treemodel [0.49377003 0.81362934 0.88394878 0.02019779 0.65950273]
forest_with_treemodel [0.66051273 0.8045093  0.82564791 0.54629215 0.83088827]
extree_with_treemodel [0.69507207 0.80789623 0.85913706 0.78432915 0.88919274]
xgb_with_treemodel [0.78588089 0.74983358 0.85401305 0.66960318 0.86299826]


# xgboost

In [18]:
kf = KFold(n_splits=5)

In [19]:
xgbreg = xgb.XGBRegressor(n_estimators=400,subsample=0.88,max_depth=3,colsample_bytree=0.88,
                             min_child_weight=2,reg_alpha=0.01,objective='reg:squarederror')

In [20]:
## forest

cross_val_score(xgbreg, X_forest, y, cv=kf, n_jobs = -1)

array([0.80542269, 0.76646104, 0.85748394, 0.70201472, 0.84473071])

In [21]:
# extree

cross_val_score(xgbreg, X_ex, y, cv=kf, n_jobs = -1)

array([0.78588089, 0.74983358, 0.85401305, 0.66960318, 0.86299826])

# svr

In [22]:
svr = svm.SVR ()

In [23]:
## forest

cross_val_score(svr, X_forest, y, cv=kf, n_jobs = -1)

array([-0.01548276, -0.03481267, -0.0145574 , -0.02346189, -0.02065488])

In [24]:
# extree

cross_val_score(svr, X_ex, y, cv=kf, n_jobs = -1)

array([-0.01548276, -0.03481267, -0.0145574 , -0.02346189, -0.02065488])

# random forest

In [25]:
cross_val_score(forest, X_ex, y, cv=kf, n_jobs = -1)

array([0.6759008 , 0.79579955, 0.8257157 , 0.54366852, 0.82925478])

# extratree

In [26]:
cross_val_score(extree, X_forest, y, cv=kf, n_jobs = -1)

array([0.69961102, 0.82107508, 0.83159497, 0.78085524, 0.89240301])

# decision tree

In [27]:
treereg = DecisionTreeRegressor()

In [28]:
## forest

cross_val_score(treereg, X_forest, y, cv=kf, n_jobs = -1)

array([0.49996661, 0.58409938, 0.70239072, 0.01620237, 0.78205544])

In [29]:
# extree

cross_val_score(treereg, X_ex, y, cv=kf, n_jobs = -1)

array([-0.00366526,  0.78486113,  0.88194242,  0.23420537,  0.70611061])

In [30]:
# pca 

cross_val_score(treereg, X_pca, y, cv=kf, n_jobs = -1)

array([-0.11350447,  0.14778862,  0.35002745,  0.73355326,  0.37124607])

In [31]:
# pca 

cross_val_score(treereg, X, y, cv=kf, n_jobs = -1)

array([0.43914011, 0.61206454, 0.69606635, 0.35090709, 0.81140548])

# 觀察降維度後受下的維度